# Tokenization

- tokenization is the step of breaking down a string into atomic units used in the model
- There're several tokenization strategies...
- Optimal splitting of words into subunits is usuallay learned from the corpus.

## Character Tokenization

- Simplest tokenization scheme, feeding each character individually to the model

In [1]:
text = 'Tokenizing text is a core taks of NLP'
tokenized_text = list(text)
print(tokenized_text)

['T', 'o', 'k', 'e', 'n', 'i', 'z', 'i', 'n', 'g', ' ', 't', 'e', 'x', 't', ' ', 'i', 's', ' ', 'a', ' ', 'c', 'o', 'r', 'e', ' ', 't', 'a', 'k', 's', ' ', 'o', 'f', ' ', 'N', 'L', 'P']


Models expect each character to be converted to an integer, this process called *Numericalization*

- One simple way to do this is by encoding each unqiue token with a unique integer.

In [2]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

{' ': 0, 'L': 1, 'N': 2, 'P': 3, 'T': 4, 'a': 5, 'c': 6, 'e': 7, 'f': 8, 'g': 9, 'i': 10, 'k': 11, 'n': 12, 'o': 13, 'r': 14, 's': 15, 't': 16, 'x': 17, 'z': 18}


In [3]:
# Transform tokenized_text according above mapping

input_ids = [token2idx[token] for token in tokenized_text]
print(input_ids)

[4, 13, 11, 7, 12, 10, 18, 10, 12, 9, 0, 16, 7, 17, 16, 0, 10, 15, 0, 5, 0, 6, 13, 14, 7, 0, 16, 5, 11, 15, 0, 13, 8, 0, 2, 1, 3]


Last step is to convert input_ids to a 2D tensor of one-hot vectors.

- **one hot** vectors are frequently used in ML to encode categorical data, which can be either ordinal or nomial

In [5]:
import pandas as pd

# Example
categorical_df = pd.DataFrame({'Name': ['Bumblebee', 'Optimus Prime', 'Megatron'], 'Label ID': [0, 1, 2]})
categorical_df

,Name,Label ID
0,Bumblebee,0
1,Optimus Prime,1
2,Megatron,2


The problem of this approach is this creates an order between the names, and models can learn this kind of relationships.

- Hence, we can create new columns for each category and assing 1 where the category is true, and a 0 otherwise.

In [6]:
# Pandas we can use get_dummies()
pd.get_dummies(categorical_df['Name'])

,Bumblebee,Megatron,Optimus Prime
0,1,0,0
1,0,0,1
2,0,1,0


In [10]:
# In Pytorch
import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids) # converts to a tensor
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
one_hot_encodings.shape

/tmp/ipykernel_1024124/2673273441.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids) # converts to a tensor


torch.Size([37, 19])

In [11]:
# verifying the idea
print(f'Token: {tokenized_text[0]}')
print(f'Tensor index: {input_ids[0]}')
print(f'One-hot: {one_hot_encodings[0]}')

Token: T
Tensor index: 4
One-hot: tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


Character-level tokenization ignores any structure in the text an treats the whole string as stream of characters.
- This helps to deal with misspelling and rare words
- Drawback: requires significant compute as linguistics structurs (such as words) need to be learned from the data.

Instead of this, we need to preserve some structure of the text is preserved during the tokenization step.
- Hence Word Tokenization is being used most of the time.

## Word Tokenization

- Split the text into words and map each word to an integer.

In [12]:
# using whitespace to tokenize the text
tokenized_text = text.split()
print(tokenized_text)

['Tokenizing', 'text', 'is', 'a', 'core', 'taks', 'of', 'NLP']


some word tokenizers have extra rules for punctuation.
- one can apply stemming or lemmatization, which normalize words to their stem, at the expense of losing some information of text
- Ex: `greater`, `great` become `great`

## Subword Tokenization

- Combines best aspects of both character and word tokenizers
- we want to split rare words into smaller units to allow the model to deal with complex words and misspellings.
- Aslo we want to keep frequent words as unique entities so that we can keep the length of our inputs to a manageable size.
- Main feature:
    - subword tokenizer is learned from the pretraining corpus using a mix of statistical rules and algorithms.
    - WordPiece one of algorithm that is being used in BERT and DistilBERT tokenizers.

In [17]:
# Tokenizer: WordPiece
from transformers import AutoTokenizer

model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [19]:
encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 19204, 6026, 3793, 2003, 1037, 4563, 27006, 2015, 1997, 17953, 2361, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Here also words are mapped to unique integers. we can convert them back to tokens as following"

In [20]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##izing', 'text', 'is', 'a', 'core', 'tak', '##s', 'of', 'nl', '##p', '[SEP]']


- prefix in '##izing' and '##p' means that the preceding string is not whitespace
- Any token with this prefix should be merged with the previous token when we convert tokens back to a string.

In [21]:
# Converting tokens into string format
print(tokenizer.convert_tokens_to_string(tokens))

[CLS] tokenizing text is a core taks of nlp [SEP]


In [22]:
# Useful methods in AutoTokenizer class
print(f'Vocab_size: {tokenizer.vocab_size}')
print(f'Model"s maximum context size: {tokenizer.model_max_length}')

Vocab_size: 30522
Model"s maximum context size: 512


In [23]:
# Names of the fields that the model expecets in the forward pass
tokenizer.model_input_names

['input_ids', 'attention_mask']

In [24]:
import tokenizers